# Providing Context:
In 2021, a survey was conducted on TikTok to promote salary transparency and assist college students and employees in exploring career options and earning potential. Participants were asked to enter their salary and other information in a shared Google Sheets document, resulting in over 80,000 entries. Because this was an unstructured survey, answers lacked defined responses and standardized formatting. The survey contained irrelevant,incomplete responses and misspellings posing challenges for data cleaning and ensuring data integrity. Analyzing the data required extensive manual effort, including coding, categorizing, and interpreting responses.

The responses to the survey were first processed in an SSIS package and loaded and then grouped into age ranges which was then loaded respectively into SQL tables and separate CSV files to speed up processing time.

This notebook focuses on cleaning the data to prepare for visualization and machine learning models that will be performed in other notebooks. This is to help with readability.

# **Data Exploration, Analyis and Cleaning:**
To tackle this project, I'll follow a structured approach, dividing the work into several steps to ensure clarity and effectiveness. Here's my roadmap:

1.	Data Loading and Initial Exploration: Load the survey data to understand the structure, identify any apparent issues, and get a sense of the cleaning required.
2.	Data Cleaning and Standardization: Clean the survey data by addressing missing values, correcting misspellings, standardizing categorical columns, and ensuring data types are appropriate and consistent for analysis.
3.	Categorization of all columns to standardize and preprocess them for machine learning modles. This will be broken down by column given the amount of processing each column requires.

**For categorization:**
1. Rename `Highest level of Education`, correct misspellings and group `Education` into categories.
2. Reduce redundancy by group the `Industry` and `Job Title` columns. This will require some web scrapping and manual mapping.
3. Correct and standardize misspellings in the `Country` column. 
4. Then Convert all salary columns into USD based on country's conversion rate to ensure accuracy and consistency. Currency conversion rates and country codes will be gotten through web scrapping as well.
5. Finalize by grouping `Gender` column.


In [1]:
import pandas as pd
pd.set_option('display.width', 1000)

files = ['SalarySur_18-21.csv', 'SalarySur_22-25.csv', 'SalarySur_26-29.csv',
         'SalarySur_30-33.csv', 'SalarySur_34-37.csv', 'SalarySur_38-41.csv',
         'SalarySur_42-49.csv', 'SalarySurv_58.csv', 'SalarySur_54-57.csv',
         'SalarySur_50-53.csv']

df = pd.concat([pd.read_csv(file) for file in files], ignore_index=True)

# Step 1: data loading and initial exploration

**Data Dictionary**

The survey data is successfully loaded, revealing the structure and some initial insights. Here's a summary of the dataset's columns:

•	`Age Range`: Categorical, representing the participant's age group.

•	`Years of Experience`: Numerical, indicating how many years of professional experience the participant has.

•	`Industry`: Categorical, showing the industry in which the participant works.

•	`Job Title`: Categorical, listing the participant's job title.

•	`Highest Level of Education Received`: Categorical, detailing the highest level of education the participant has completed.

•	`Country`: Categorical, specifying the country of the participant's employment.

•	`Annual Salary`: Numerical, the participant's annual salary.

•	`Annual Bonus`: Numerical, the annual bonus amount the participant receives, if any.

•	`Signon Bonus`: Numerical, the sign-on bonus amount the participant received upon hiring, if any.

•	`Gender`: Categorical, indicating the participant's gender.


In [2]:
#displays first 5 rows
df.head() , len(df)

(  Age Range Years of Experience        Industry                      Job Title                Highest Level of Education Received      Country  Annual Salary  Annual Bonus  Signon Bonus  Gender
 0     18-21                   1      Consulting                 Office Manager                                  Bachelor's Degree       CANADA        42000.0           0.0           0.0  Female
 1     18-21                   1          Retail           Account Coordinator                                   Bachelor's Degree          USA        37000.0        1200.0           0.0  Female
 2     18-21                   1  Transportation  Engineering product manager 1                                  Bachelor's Degree          USA        75000.0           0.0        3000.0  Female
 3     18-21                   1      Accounting       Junior credit controller                       Some High School, No Diploma      ENGLAND        19500.0         250.0           0.0  Female
 4     18-21             

**Missing Values:**

There are missing entries across multiple columns, with "Industry" and "Job Title" having the most significant number of missing values (726 and 792, respectively). Other columns with missing data include "Years of Experience", "Annual Salary", "Annual Bonus", "Signon Bonus", and a few others.

**Summary Statistics:**

These statistics provide a snapshot of the salary, annual bonus, and sign-on bonus distributions within the data set. the data suggests a varied distribution of salaries, annual bonuses, and sign-on bonuses, with a significant number of individuals not receiving bonuses or sign-on amounts. Additionally, there are outliers with exceptionally high salaries and bonuses. Further analysis could explore factors such as age range, years of experience, industry, job title, education level, country, and gender to understand their influence on salary and bonus distributions.

From the statistics, we can infer that the majority of individuals in the dataset receive annual salaries ranging from $52,000 to $95,000, with a median salary of $70,000. Additionally, most reported annual bonuses and sign-on bonuses are zero or relatively low, with some extreme outliers indicating a small percentage of individuals receiving exceptionally high bonuses.

In [3]:
# Check for missing values in the dataset
missing_values = df.isnull().sum()

# Summary statistics for numerical columns to identify any inconsistencies
summary_statistics = df.describe()

print(missing_values, summary_statistics)

Age Range                                1
Years of Experience                     10
Industry                               726
Job Title                              792
Highest Level of Education Received      1
Country                                  1
Annual Salary                           61
Annual Bonus                             3
Signon Bonus                            19
Gender                                   1
dtype: int64        Annual Salary   Annual Bonus   Signon Bonus
count   48233.000000   48291.000000   48275.000000
mean    81582.805945    6447.878038    1482.807171
std     56721.869219   21894.745465    9507.873636
min         0.000000       0.000000       0.000000
25%     52000.000000       0.000000       0.000000
50%     70000.000000       0.000000       0.000000
75%     95000.000000    5000.000000       0.000000
max    990000.000000  750000.000000  715000.000000


**Detecting and handling outliers:**

Overall, outlier detection using the IQR method serves as an important step in data preprocessing, contributing to improved data quality, robust statistical analyses, and better-performing machine learning models.

The results indicate that there are 2461 outliers in the 'Annual Salary' column, 5979 outliers in the 'Annual Bonus' column, and 5875 outliers in the 'Signon Bonus' column. Upon further exploration, the outliers in the annual salaries are extermely high salary rates and the outliers in Annual Bonus and Signon Bonus are mostly zeros with some high bonuses. 

Employing IQR to handle missing values will result in loss of valuable information, especially since the missing values are not randomly distributed across the dataset. Using the IQR method to handle missing salary values doesn't take into account these dependencies. It treats each salary value in isolation, potentially leading to imprecise estimations that do not reflect the true distribution of salaries within different job titles or industries. Dropping rows based on the presence of missing values could lead to biased analyses or inaccurate modeling. 

In [4]:
# Outlier Detection using IQR
Q1 = df[['Annual Salary', 'Annual Bonus', 'Signon Bonus']].quantile(0.25)
Q3 = df[['Annual Salary', 'Annual Bonus', 'Signon Bonus']].quantile(0.75)

IQR = Q3 - Q1

# Calculating IQR Define bounds for outliers.
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Count outliers
outlierscount = ((df[['Annual Salary', 'Annual Bonus', 'Signon Bonus']] < lower_bound) | 
            (df[['Annual Salary', 'Annual Bonus', 'Signon Bonus']] > upper_bound)).sum()

# Identify outliers
outliers = ((df[['Annual Salary', 'Annual Bonus', 'Signon Bonus']] < lower_bound) | 
            (df[['Annual Salary', 'Annual Bonus', 'Signon Bonus']] > upper_bound))

# Print outliers
print("Outliers in Annual Salary:")
print(outlierscount)

# Filter DataFrame to include only rows with outliers
outlier_rows = df[outliers.any(axis=1)]
print(outlier_rows)

Outliers in Annual Salary:
Annual Salary    2553
Annual Bonus     6134
Signon Bonus     6017
dtype: int64
      Age Range Years of Experience            Industry                      Job Title Highest Level of Education Received Country  Annual Salary  Annual Bonus  Signon Bonus      Gender
2         18-21                   1      Transportation  Engineering product manager 1                   Bachelor's Degree     USA        75000.0           0.0        3000.0      Female
9         18-21                   1          Accounting                    Assurance 1                   Bachelor's Degree  CANADA        55000.0           0.0        2000.0        Male
10        18-21                   1          Accounting                Audit Associate                     Master's Degree     USA        63500.0           0.0        3000.0      Female
11        18-21                   1          Accounting                       Sale Rep                   Bachelor's Degree     USA        50000.0     

Let's first address Missing Values and decide how they'll be handled: 

Given the substantial number of the data set, and large number of null values in the industry and job titles column, the best approach is to drop the null values for simplification. Data set went from 48292 rows to 46889

In [5]:
# Dropping rows with any missing values
df = df.dropna()

#verify nulls have been removed
df.isnull().sum(), print(len(df)), df.info()

46889
<class 'pandas.core.frame.DataFrame'>
Int64Index: 46889 entries, 0 to 48292
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Age Range                            46889 non-null  object 
 1   Years of Experience                  46889 non-null  object 
 2   Industry                             46889 non-null  object 
 3   Job Title                            46889 non-null  object 
 4   Highest Level of Education Received  46889 non-null  object 
 5   Country                              46889 non-null  object 
 6   Annual Salary                        46889 non-null  float64
 7   Annual Bonus                         46889 non-null  float64
 8   Signon Bonus                         46889 non-null  float64
 9   Gender                               46889 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.9+ MB


(Age Range                              0
 Years of Experience                    0
 Industry                               0
 Job Title                              0
 Highest Level of Education Received    0
 Country                                0
 Annual Salary                          0
 Annual Bonus                           0
 Signon Bonus                           0
 Gender                                 0
 dtype: int64,
 None,
 None)

# Step 2: Data Cleaning, Standardizing and Categorization:
**Standardizing columns for uniformity**

**Categorical columns to be standardized:** Highest level of Education, Industry, Job title, Country, Salary columns, Gender in this order.

As mentioned at the beginning, categorization will be done in steps:
1. Rename 'Highest level of Education', correct misspellings and group Education into categories.
2. categorizing 'Industry' This will require some web scrapping and manual mapping.
3. Repeat for 'Job Title' columns. 
3. Correct and standardize misspellings in the 'Country' column. 
4. Then Convert all salary columns into USD based on country's conversion rate to ensure accuracy and consistency. Currency conversion rates and country codes will be gotten through web scrapping as well.
5. Finalize by grouping 'Gender' column.

# 1. Categorizing 'Highest level of Education' column

Renaming the column, correcting mispellings and categorizing values into the specified categories using a function.

In [6]:
#Rename "Highest Level of Education Received" to "Education"
df.rename(columns={'Highest Level of Education Received': 'Education'}, inplace=True)

# Define a function to categorize the education levels
def categorize_education(level):
    #level = level.strip()
    #detailed categories and keywords to match them
    education_mappings = {
        "No Schooling": ['Some High School, No Diploma','SomeHighSchool,NoDiploma','No Schooling Completed','NoSchoolingCompleted',"N/a"] ,
        "Trade School": ['Trade School', 'Trade,Technical,VocationalTraining','Trade, Technical, Vocational Training'] ,
        "Some college": ["SomeCollegecredit,nodegree","Some College credit, no degree"],
        "High School Diploma": ["HighSchoolGraduate,Diplomaortheequivalent(e.g.GED)","High School Graduate, Diploma or the equivalent (e"] ,
        "Associate Degree":['AssociateDegree', 'Associate Degree'] ,
        "Bachelor's Degree": ['BachelorsDegree',"Bachelor's Degree"] ,
        "Master's Degree": ['MastersDegree',"Master's Degree" ],
        "Doctoral Degree": ['Doctorate Degree', 'DoctorateDegree',"Professional Degree","ProfessionalDegree"] ,
        
    }
    for category, keywords in education_mappings.items():
        if any(keyword in level for keyword in keywords):
            return category
    return "Other" # For any education levels not explicitly mapped

# Apply the categorization function to the "Education" column
df['Education'] = df['Education'].apply(categorize_education)

df['Education'].unique()

array(["Bachelor's Degree", 'No Schooling', 'High School Diploma',
       "Master's Degree", 'Associate Degree', 'Some college',
       'Trade School', 'Doctoral Degree'], dtype=object)

# 2. Categorizing Industry Column:

steps:

1. Webscrape to get a list of all industries from bls.gov and add to list that was mapped manually.
2. Remove duplicate values by stripping trailing spaces and converting all entries to lower case
3. using fuzzy matching to map the entries with an 80% match and other for non matches.

1. Getting List of Industries from bls.gov to use for Fuzzy matching:

This script sends a request https://www.bls.gov/iag/tgs/iag_index_alpha.htm to extract the list of industry names and save them to the file industry_titles.txt which will be used to further standardize the column, by grouping similar entries together. The function iterates through all sections containing industry titles and extracts them. Each extracted industry title undergoes the same splitting process to exclude any additional information.

Categorizing and standardizing "Country" Column

Correct the misspellings in the "Country" column and group similar countries together.

In [7]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def scrape_industry_titles(url):
    print("Attempting to establish connection...")
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://www.bls.gov/iag/tgs/iag_index_alpha.htm'
    }

    try:
        response = requests.get(url, headers=headers, timeout=60)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print("Failed to retrieve industry titles:", str(e))
        return None

    print("Connection succeeded. Parsing HTML content...")
    soup = BeautifulSoup(response.content, 'html.parser')
    industry_section = soup.find('div', class_='iag--tgs')

    if not industry_section:
        print("Industry section not found.")
        return None

    # Extract industry titles
    industry_titles = [title_element.text.strip().split(" (N")[0].strip() for title_element in industry_section.find_all('li')]
    print("Scraping complete.")
    return industry_titles

def save_to_csv(industry_titles, filename):
    with open(filename, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Industry Titles"])  # header
        writer.writerows([[title.lower().strip()] for title in industry_titles])

if __name__ == "__main__":
    url = 'https://www.bls.gov/iag/tgs/iag_index_alpha.htm'
    industry_titles = scrape_industry_titles(url)

    if industry_titles:
        print("Number of Industries:", len(industry_titles))
        print("\nIndustry Titles:")
        for title in industry_titles[:5]:
            print(title)

        filename = 'industry_titles.csv'
        save_to_csv(industry_titles, filename)
        print(f"\nIndustry titles saved to '{filename}'.")

    time.sleep(1)


Attempting to establish connection...
Connection succeeded. Parsing HTML content...
Scraping complete.
Number of Industries: 115

Industry Titles:
Accommodation
Accommodation and Food Services
Administrative and Support Services
Administrative and Support and Waste Management and Remediation Services
Agriculture, Forestry, Fishing and Hunting

Industry titles saved to 'industry_titles.csv'.


2. Remove duplicate values by stripping trailing spaces and converting all entries to lower case

Industries: Standardizing formatting reduced entries from 8,424 down to 6,570.

Efficiency: With the Industry column already cleaned and deduplicated, the fuzzy matching operation becomes significantly more efficient.

In [8]:

all_industries = len(df['Industry'].unique())

# Standardize formatting for "Industry" column
df['Industry'] = df['Industry'].str.lower().str.strip()
# After standardization, let's re-check the unique values for a sample comparison
unique_industries = df['Industry'].unique()


print("Total industries: ", all_industries)
print("Unique industries: ", len(unique_industries))
print("First 5 unique industries: ", unique_industries[:5])

Total industries:  8424
Unique industries:  6487
First 5 unique industries:  ['consulting' 'retail' 'transportation' 'accounting' 'advertising']


3. Apply fuzzy wuzzy to group entries:
115 industries were extracted and over 88 were individually mapped. Further cleaning was done in Excel increasing the number of job titles from 115 to 296 to provide variety in mapping.
Categorization Strategy:

1. Fuzzy Matching: Given the potential for misspellings and variations, employing a technique like fuzzy matching can help match the survey entries to the cleaned lists of industries.
2. Manual Review and Correction: Depending on the effectiveness of fuzzy matching, some manual review may be required for entries with low matching scores.

In [9]:
from thefuzz import process

# Load the industry_titles.csv
with open('industry_titlesALL.csv', 'r') as file:
    categories = file.read().splitlines()[1:]  # Remove header

# Define a function for fuzzy matching
def fuzzy_match_industries(unique_industries, categories):
    matched_categories = {
        industry: process.extractOne(industry, categories)[0]  # Get the best match
        for industry in unique_industries
        if not pd.isna(industry)  # Skip NaN values
    }
    
    return matched_categories

# Extract unique industries from the DataFrame
unique_industries = df['Industry'].unique()

# Apply fuzzy matching to unique industries
matched_categories = fuzzy_match_industries(unique_industries, categories)

# Map the entire DataFrame using this dictionary
df['Industry'] = df['Industry'].map(matched_categories).fillna("Other").str.title()

print(df['Industry'])

0                      Consulting
1                          Retail
2              Air Transportation
3                      Accounting
4                      Accounting
                   ...           
48288                     Biotech
48289    Computer And Electronics
48290                    Plumbing
48291               Manufacturing
48292                     Finance
Name: Industry, Length: 46889, dtype: object


# 3. Categorizing Job Title Column:

follow the same steps performed for the industry column:

1. Webscrape to get a list of all jobs from bls.gov and add to list that was mapped manually.
2. remove duplicate values by stripping trailing spaces and converting all entries to lower case
3. using fuzzy matching to map the entries with an 80% match and other for non matches.

1. **Getting List of Job Titles from bls.gov to use for Fuzzy matching:**

This script sends a request https://www.bls.gov/iag/tgs/iag_index_alpha.htm to extract the list of industry names and save them to the file industry_titles.csv which will be used to further standardize the column, by grouping similar entries together. The function iterates through all sections containing job titles and extracts job titles from each of them. Each extracted job title undergoes the same splitting process to exclude any additional information.

In [10]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def scrape_job_titles(url):
    print("Attempting to establish connection...")
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://www.bls.gov/ooh/a-z-index.htm#'
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print("Failed to retrieve job titles:", str(e))
        return None

    print("Connection succeeded. Parsing HTML content...")
    soup = BeautifulSoup(response.content, 'html.parser')
    job_sections = soup.find_all('div', class_='a-z-list-box')

    job_titles = []
    for job_section in job_sections:
        job_titles.extend([title_element.text.strip().split(", see:")[0].strip() for title_element in job_section.find_all('li')])

    print("Scraping complete.")
    return job_titles

def save_to_csv(job_titles, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows([title.lower().strip()] for title in job_titles)
        
if __name__ == "__main__":
    url = 'https://www.bls.gov/ooh/a-z-index.htm#'
    job_titles = scrape_job_titles(url)

    if job_titles:
        print("Number of Job Titles:", len(job_titles))
        print("\nJob Titles:")
        for title in job_titles[:5]:
            print(title)

        filename = 'job_titles.csv'
        save_to_csv(job_titles, filename)
        print(f"\nJob titles saved to '{filename}'.")

    time.sleep(1)

Attempting to establish connection...


Connection succeeded. Parsing HTML content...
Scraping complete.
Number of Job Titles: 4763

Job Titles:
21 Dealer
3D Animator
3rd Grade Reading Teacher
4th Grade Math Teacher
911 Dispatcher

Job titles saved to 'job_titles.csv'.


2. **Remove duplicate values by stripping trailing spaces and converting all entries to lower case**

**Job Titles:** standardizing formatting reduced entries from 20,583 down to 15,717.

**Efficiency:** With the "Job Title" already cleaned and deduplicated, the fuzzy matching operation becomes significantly more efficient.

In [11]:
# Count the total number of unique job titles
all_jobs = len(df['Job Title'].unique())

# Standardize formatting for "Industry" and "Job Title" columns
df['Job Title'] = df['Job Title'].str.lower().str.strip()

# Check the unique job titles after standardization
unique_job_titles = df['Job Title'].unique()

# Print the total number of unique job titles, before and after standardization
print("Total job titles:", all_jobs)
print("Number of unique job titles after standardization:", len(unique_job_titles))
print("Unique job titles:", unique_job_titles)

Total job titles: 20361
Number of unique job titles after standardization: 15717
Unique job titles: ['office manager' 'account coordinator' 'engineering product manager 1'
 ... 'diagnostic imaging engineer'
 'certified regulatory compliance manager' 'regional account manager']


3. **Apply fuzzy wuzzy to group entries:**

Over 4500 job titles were extracted from bls which were then combined with a lists of job titles that were manually mapped(88). Further cleaning was done in Excel reducing the number of job titles significnatly from 5000 to 2500 to help facilitate and simplify the fuzzy match process. Still, with over 2500 job titles to match agaisnt, a different approach needs to taken:

1. **Parallel Processing:** Use parallel processing libraries like joblib to distribute the fuzzy matching workload across multiple CPU cores. By setting n_jobs= -1 in Parallel, we utilize all available CPU cores, speeding up the matching process.
2. **Batch Processing:** Break down the list of unique job titles into smaller batches and perform fuzzy matching on these batches. This can help manage memory usage and improve responsiveness.
3. **Manual Review and Correction:** Depending on the effectiveness of fuzzy matching, some manual review may be required for entries with low matching scores.

In [12]:
from thefuzz import process
import pandas as pd
from joblib import Parallel, delayed

# Standardize job titles in the DataFrame
df['Job Title'] = df['Job Title'].str.lower().str.strip()

# Load and clean job titles from CSV
with open('job_titlesALL.csv', 'r', encoding='latin-1') as file: 
    job_titles = file.read().splitlines()[1:] # Remove header

#Function to fuzzy match job titles
def fuzzy_match_job(job, job_list):
    best_match, score = process.extractOne(job, job_list)
    return best_match if score > 73 else "Other"

# Fuzzy match unique job titles in parallel
matched_jobs = Parallel(n_jobs=-1)(delayed(fuzzy_match_job)(title, job_titles) for title in df['Job Title'].unique())

# Create a dictionary of matched job titles
matched_jobs_dict = dict(zip(df['Job Title'].unique(), matched_jobs))

# Map the job titles in the DataFrame using the matched dictionary
df['Job Title'] = df['Job Title'].map(matched_jobs_dict).fillna("Other").str.title()

print(df['Job Title'])

0        Business Office Manager
1            Account Coordinator
2                       Engineer
3                     Controller
4                     Accountant
                  ...           
48288                         Vp
48289                   Engineer
48290                      Other
48291                        Ceo
48292                    Manager
Name: Job Title, Length: 46889, dtype: object


# Verifying "Industry" and "Job Title" column mappings

The `Industry` column went from 6,487 unique entries to 173 unique entries after the standardization.

The `Job Title` column went from 15,717 unique entries to 1,115 unique entries after the standardization.

In [13]:
print("Current number of unique Industries:", len(df['Industry'].unique()))

print("Current number of unique Job Titles:" ,len(df['Job Title'].unique()))


Current number of unique Industries: 173
Current number of unique Job Titles: 1115


# 4. Standardizing 'Country' Column: 
1. Inspect column
2. clean and standardize. Group if necessary.


In [14]:
# We will first inspect the unique values to identify the corrections needed
df['Country'].unique()

array(['CANADA', 'USA', 'ENGLAND', 'NETHERLANDS', 'AUSTRALIA',
       'NEW ZEALAND', 'IRELAND', 'TURKEY ', 'SRILANKA', 'SINGAPORE',
       'PHILIPPINES', 'ENGLAND ', 'SOUTH KOREA', 'ICELAND', 'SCOTLAND ',
       'MEXICO', 'MONACO', 'LITHUANIA', 'NEW ZEALAND ', 'NORWAY', 'QATAR',
       'IRELAND ', 'GERMANY', 'SPAIN', 'SINGAPORE ', 'ARGENTINA',
       'FINLAND', 'JAPAN', 'FRANCE', 'AFGHANISTAN', 'SWEDEN', 'BELIZE',
       'SCOTLAND', 'RUSSIA', 'SAUDI ARABIA ', 'BANGLADESH', 'UAE',
       'SWITZERLAND', 'POLAND', 'GERMANY ', 'CHILE ', 'HUNGARY ',
       'PORTUGAL', 'MOROCCO', 'BELGIUM', 'THAILAND ', 'CAMBODIA ',
       'DENMARK', 'NETHERLANDS ', 'MALAYSIA', 'LITHUANIA ',
       'CAYMAN ISLANDS', 'CHILE', 'COLOMBIA', 'PERU', 'PUERTO RICO',
       'ISRAEL', 'ALBANIA', 'BRAZIL', 'SWITZERLAND ', 'SERBIA', 'LATVIA',
       'GUATEMALA', 'SOUTH AFRICA', 'EGYPT', 'SPAIN ', 'ITALY ',
       'POLAND ', 'BULGARIA', 'DOMINICAN REPUBLIC ', 'BULGARIA ',
       'ROMANIA', 'PANAMÁ', 'AFGHANISTAN ', 'BAH

2. **Standardizing column:**

Having inspected the unique values in the `Country` column, they are a few issues to correct, such as trailing spaces and variations in country names that should be grouped together. For example, "USA" and variations like "united states of america" should all be standardized to "USA". The list also includes "ENGLAND" and "SCOTLAND", which can be grouped under "UK" for simplicity, along with any other variations like "britain".

Also, correcting mispellings and formatting at this stage will to get conversion rates per country to accurately calculate all salaries in USD.

In [15]:
# Correct and group country names

def standardize_country_names(country):
    country = country.strip().upper()  # Normalize case and strip whitespace
    # Map to group countries
    country_mapping = {
        'UNITED STATES': 'USA',
        'UNITED KINGDOM': ['ENGLAND', 'BRITAIN', 'ENGLANDRAINE', 'SCOTLAND'],
        'SWITZERLAND': 'SWITZERAND',
        'UNITED ARAB EMIRATES': 'UAE',
        'CANADA': 'BRITISH COLUMBIA',
        "SRI LANKA": "SRILANKA",
        "HONDURAS": "HONDRUAS",
        "TRINIDAD AND TOBAGO": "TRINIDAD & TOBAGO",
        "PANAMA": ['PANAMA', 'PANAMÁ']
        # Add more mappings if necessary
    }

    for key, value in country_mapping.items():
        if country in value:
            return key
    
    return country

# Apply the function to the "Country" column
df['Country'] = df['Country'].apply(standardize_country_names).str.title()
df['Country'].unique()

array(['Canada', 'United States', 'United Kingdom', 'Netherlands',
       'Australia', 'New Zealand', 'Ireland', 'Turkey', 'Sri Lanka',
       'Singapore', 'Philippines', 'South Korea', 'Iceland', 'Mexico',
       'Monaco', 'Lithuania', 'Norway', 'Qatar', 'Germany', 'Spain',
       'Argentina', 'Finland', 'Japan', 'France', 'Afghanistan', 'Sweden',
       'Belize', 'Russia', 'Saudi Arabia', 'Bangladesh',
       'United Arab Emirates', 'Switzerland', 'Poland', 'Chile',
       'Hungary', 'Portugal', 'Morocco', 'Belgium', 'Thailand',
       'Cambodia', 'Denmark', 'Malaysia', 'Cayman Islands', 'Colombia',
       'Peru', 'Puerto Rico', 'Israel', 'Albania', 'Brazil', 'Serbia',
       'Latvia', 'Guatemala', 'South Africa', 'Egypt', 'Italy',
       'Bulgaria', 'Dominican Republic', 'Romania', 'Panama', 'Bahrain',
       'Honduras', 'Maldives', 'Jamaica', 'Taiwan', 'China', 'Estonia',
       'Ecuador', 'Kenya', 'Trinidad And Tobago', 'Paraguay', 'Pakistan',
       'India', 'El Salvador', 'Hong 

# 5. Converting "Annual Salary", "Annual Bonus" and "Annual Signon Bonus" to USD based on participant country.

It is to be assumed that particpants entered their salaries based upon the country in which they reside.

**Categorizing All Salary Columns. A 3 step approach:**
1. Verify that country names in the `Country` column match how they're listed on 
2. Fetch for currendy codes for the unique list of countries in our data frame
3. Fetch exchange rates using country codes we got in step 2.
4. Use the exchange rates and create a function that converts `Annual Salary`, `Annual Bonus` and `Signon Bonus` columns to USD.

Performing all the steps above allows for consistent and accurate salary predictions for all countries based on USD. If not, an entry for South Africa within the data set has a an annual salary for 380,000 which is approximately (SAR 10,889,627.76) which is grossly inaccurate based on research showing the median to highest salary for an engineer in South African is between (ZAR 450K - 1.4M). If the salary is not converted to USD, that would make S.A. amongst the highest salaries which skews the results. After the salary conversions, the ne salary becomes 20814.30

In [16]:
# Choose a specific country to view the annual salary before conversion
desired_countries = ['South Africa'] #'China','South Korea','Ireland', 

# Filter the DataFrame for rows where the 'Country' column matches any of the desired countries
filtered_df = df[df['Country'].isin(desired_countries)]
print(filtered_df[:5] )
df['Annual Salary'].isnull().sum()

      Age Range Years of Experience                     Industry            Job Title        Education       Country  Annual Salary  Annual Bonus  Signon Bonus  Gender
13618     22-25                   3                   Accounting              Auditor  Doctoral Degree  South Africa      390000.00       40000.0           0.0    Male
14672     22-25                   3  Rental And Leasing Services              Analyst  Master's Degree  South Africa      461309.99           0.0           0.0  Female
19075     22-25                   5              Branding/Design     Graphic Designer     No Schooling  South Africa       54000.00           0.0           0.0    Male
27678     26-29                   5                  Health Care  Account Coordinator     Some college  South Africa       53000.00           0.0           0.0  Female
39123     30-33                   8                      Biotech                Other     Some college  South Africa       40000.00           0.0           0.0 

0

1. **Verify the countries in the "Country" column are named properly.** 

The function will return a dictionary with country names as keys and their currency codes as values. For countries with multiple currencies, only the first currency code listed is returned. This dictionary is is then stored to a file and used to standardize the country names in your DataFrame to match the API's format.

With this dictionary, we can cross-reference the country names in the `Country` column and correct any discrepancies. This will help ensure that all subsequent API calls for currency codes based on country names are successful.

In [17]:
import requests
import csv

def get_all_countries_currencies():
    """
    Fetch all country names and their corresponding currency codes from the REST Countries API.
    :return: Dictionary mapping country names to currency codes.
    """
    try:
        # REST Countries API endpoint for all countries
        response = requests.get("https://restcountries.com/v3.1/all")
        data = response.json()
        # Dictionary comprehension to extract countries and their currencies
        countries_currencies = {country['name']['common']: list(country['currencies'].keys())[0]
                                for country in data if 'currencies' in country}
        return countries_currencies
    except Exception as e:
        print(f"Error fetching countries and currencies: {e}")
        return {}

# Fetch all countries and currencies
all_countries_currencies = get_all_countries_currencies()

# Write the data to a CSV file
with open('country_currency_codes.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Country', 'Currency Code'])  # Write header
    for country, currency in all_countries_currencies.items():
        writer.writerow([country, currency])


# Preview the first 5 entries
preview = list(all_countries_currencies.items())[:10]
print(preview)

[('Cyprus', 'EUR'), ('Eritrea', 'ERN'), ('Liberia', 'LRD'), ('Bermuda', 'BMD'), ('Vatican City', 'EUR'), ('Cook Islands', 'CKD'), ('Somalia', 'SOS'), ('Zambia', 'ZMW'), ('Venezuela', 'VES'), ('Turkmenistan', 'TMT')]


2. **Fetch for currendy codes for the unique list of countries in the data frame.**


This code defines a function `get_country_currency_mapping(countries)` that takes a list of country names as input. It then iterates over each country in the input list, makes a request to the REST Countries API to fetch data about the country, extracts the currency code from the response, and stores it in a dictionary mapping each country name to its corresponding currency code. If an error occurs during the process, it prints an error message.

I manually set the currency code for Ireland because it kept getting GDP which is incorrect.

This will be used to extract conversion rates to calculate the salaries in the following section

In [18]:
import requests

def get_country_currency_mapping(countries):
    """
    Fetch currency codes for a given list of countries using the REST Countries API.
    :param countries: List of country names.
    :return: Dictionary mapping country names to currency codes.
    """
    country_currency = {}
    for country in countries:
        try:
            if country == "Ireland":  # Check for Ireland and assign the correct currency code
                country_currency[country] = "EUR"
            else:
                # REST Countries API endpoint; adjust as necessary
                response = requests.get(f"https://restcountries.com/v3.1/name/{country}")
                data = response.json()
                # Extracting the first currency code for the country
                currency_code = list(data[0]['currencies'].keys())[0]
                country_currency[country] = currency_code
        except Exception as e:
            print(f"Error fetching currency for {country}: {e}")
    return country_currency

# Example usage with a list of unique countries from your dataset
unique_countries = df['Country'].unique().tolist()
country_currency_mapping = get_country_currency_mapping(unique_countries)

# Preview the first 5 entries
preview = list(country_currency_mapping.items())[:10]
print(preview)

[('Canada', 'CAD'), ('United States', 'USD'), ('United Kingdom', 'GBP'), ('Netherlands', 'EUR'), ('Australia', 'AUD'), ('New Zealand', 'NZD'), ('Ireland', 'EUR'), ('Turkey', 'TRY'), ('Sri Lanka', 'LKR'), ('Singapore', 'SGD')]


3. **Fetch exchange rates using country codes we got in step 2.**

The `fetch_exchange_rate` function uses `get_country_currency_mapping(unique_countries)` to extract conversion rate for each country. The `convert_salary `function then multiplies the `Annual Salary`, `Annual Bonus` and `Signon Bonus` by the rate to convert the currency to USD. The function is then applied to the columns.

This code defines a function `fetch_exchange_rate(api_key, base_currency)`responsible for retrieving the exchange rate for a given base currency against USD from the ExchangeRate-API. If the base currency is USD, it returns 1.0 since the exchange rate for USD to USD is 1.0. The function first checks if the base currency is USD. If it is, it returns 1.0. Otherwise, it constructs the API endpoint URL using the provided API key and base currency. It then sends a request to the API, retrieves the response in JSON format, and extracts the conversion rate if the request is successful. If the request fails, it prints an error message and returns None.

In [19]:
def fetch_exchange_rate(api_key, base_currency):
    """
    Fetches the exchange rate to USD for the given base currency using the ExchangeRate-API.
    Assumes 1 USD exchange rate for USD to simplify conversions.
    """
    if base_currency == "USD":
        return 1.0
    url = f"https://v6.exchangerate-api.com/v6/{api_key}/pair/{base_currency}/USD"
    response = requests.get(url)
    data = response.json()
    if data['result'] == 'success':
        return data['conversion_rate']
    else:
        print(f"Failed to fetch data for {base_currency}")
        return None

api_key = "f59d7a6da77e535013faad78"
# Ensure this uses the correct mapping between country names and their currency codes
exchange_rates = {currency: fetch_exchange_rate(api_key, currency) for currency in country_currency_mapping.values()}
print("Exchange rates: ",exchange_rates)

# Corrected function to convert salary columns to USD
def convert_salary(row, exchange_rates, country_currency_mapping):
    """this function ensures that salary data in different currencies are converted to USD, except for 
    salaries already denoted in USD. Then rounds each converted salary to 2 dec places."""
    currency_code = country_currency_mapping.get(row['Country'], 'USD')
    rate = exchange_rates.get(currency_code, 1.0)  # Default to 1.0 for USD or missing rates
    
    for col in ['Annual Salary', 'Annual Bonus', 'Signon Bonus']:
        if pd.notnull(row[col]):
            #converts the salaries to USD for all countries except the US
            row[col] = round(row[col] * rate, 2) if currency_code != 'USD' else round(row[col], 2)
    return row

# Apply the conversion with corrected logic
df = df.apply(lambda row: convert_salary(row, exchange_rates, country_currency_mapping), axis=1)

# Check the results
print(df[['Country', 'Annual Salary', 'Annual Bonus', 'Signon Bonus']].head())


Exchange rates:  {'CAD': 0.7391, 'USD': 1.0, 'GBP': 1.275, 'EUR': 1.0888, 'AUD': 0.6556, 'NZD': 0.6063, 'TRY': 0.03084, 'LKR': 0.00329, 'SGD': 0.7449, 'PHP': 0.0178, 'KRW': 0.00074829, 'ISK': 0.007299, 'MXN': 0.05971, 'NOK': 0.09414, 'QAR': 0.2747, 'ARS': 0.001172, 'JPY': 0.006614, 'AFN': 0.01402, 'SEK': 0.09585, 'BZD': 0.5, 'RUB': 0.01082, 'SAR': 0.2667, 'BDT': 0.009114, 'AED': 0.2723, 'CHF': 1.1262, 'PLN': 0.2516, 'CLP': 0.001035, 'HUF': 0.00276, 'MAD': 0.09952, 'THB': 0.02772, 'KHR': 0.00024728, 'DKK': 0.1462, 'MYR': 0.2111, 'KYD': 1.2, 'COP': 0.00025794, 'PEN': 0.2705, 'ILS': 0.2735, 'ALL': 0.01053, 'BRL': 0.199, 'RSD': 0.009267, 'GTQ': 0.1283, 'ZAR': 0.05329, 'EGP': 0.0213, 'BGN': 0.5563, 'DOP': 0.01692, 'RON': 0.2181, 'PAB': 1.0, 'BHD': 2.6596, 'HNL': 0.04053, 'MVR': 0.06486, 'JMD': 0.006499, 'TWD': 0.03143, 'KES': 0.00755, 'TTD': 0.1477, 'PYG': 0.00013681, 'PKR': 0.003589, 'HKD': 0.1278, 'SDG': 0.001877, 'LBP': 1.117e-05}
          Country  Annual Salary  Annual Bonus  Signon Bo

# 5. Finalize by grouping gender column.
1. Remove special characters (+) and Rename `Years of Experience` to `Experience`.
2. Standardize the `Gender` Column: edit descriptions to a consistent format, accounting for common variations.
3. Save all cleaned files to *Cleaned_Sal_Sur.csv* to be used for part 2 and 3 of the project.

In [20]:
#Step 1: Replace "20+" with "20" in the 'Years of Experience' column
df['Years of Experience'] = df['Years of Experience'].str.replace('20\+', '20', regex=True)

#Rename "Years of Experience" to "Experience"
df.rename(columns={'Years of Experience': 'Experience'}, inplace=True)

# Step 2: Standardize the Gender Column
def standardize_gender(gender):
    gender = gender.strip().lower()
    if 'female' in gender:
        return 'Female'
    elif 'male' in gender:
        return 'Male'
    elif 'non-binary' in gender or 'nonbinary' in gender:
        return 'Non-Binary'
    else:
        return 'Other'
#apply function to gender column
df['Gender'] = df['Gender'].apply(standardize_gender)

#Save cleaned file for ML models and visualizations.
df.to_csv('Cleaned_SalSur.csv', index=False)

df['Gender'].unique()

array(['Female', 'Male', 'Non-Binary', 'Other'], dtype=object)

In [21]:
#preview of cleaned and formatted columns
df.head(10)

,Age Range,Experience,Industry,Job Title,Education,Country,Annual Salary,Annual Bonus,Signon Bonus,Gender
0,18-21,1,Consulting,Business Office Manager,Bachelor's Degree,Canada,31042.2,0.00,0.0,Female
1,18-21,1,Retail,Account Coordinator,Bachelor's Degree,United States,37000.0,1200.00,0.0,Female
2,18-21,1,Air Transportation,Engineer,Bachelor's Degree,United States,75000.0,0.00,3000.0,Female
3,18-21,1,Accounting,Controller,No Schooling,United Kingdom,24862.5,318.75,0.0,Female
4,18-21,1,Accounting,Accountant,High School Diploma,Netherlands,32664.0,2395.36,0.0,Female
5,18-21,2,Accounting,Actuarial Associate,High School Diploma,United Kingdom,31237.5,1912.50,0.0,Female
6,18-21,1,Accounting,Staff Accountant,Master's Degree,United States,58000.0,0.00,0.0,Female
7,18-21,1,Accounting,Intern,Associate Degree,United States,44160.0,500.00,0.0,Female
8,18-21,1,Accounting,Consultant,Bachelor's Degree,United States,66600.0,0.00,0.0,Male
9,18-21,1,Accounting,Quality Assurance Analysts And Testers,Bachelor's Degree,Canada,40650.5,0.00,1478.2,Male
